## Step 1
* Collect EU leaderboards.<br>
* Collect Parent realm IDs (since only parent realms return leaderboards in API for whatever reason).<br>
* Collect all Protection Paladins<br>
* Save results in CSV for Step 2; Gear Collection<br>

In [1]:
import pandas as pd
import requests

In [2]:
# Create a new Access Token
def create_access_token(client_id, client_secret, region = 'eu'):
    data = { 'grant_type': 'client_credentials' }
    response = requests.post('https://%s.battle.net/oauth/token' % region, data=data, auth=(client_id, client_secret))
    return response.json()
response = create_access_token('59cc7fe2d633479a8eabf7bfd61ec8aa', 'fq48ndwEJ6P7dLMxG7gTThGlYwIPVjmD')
token = response['access_token']

In [ ]:
def eu_realms():
    search = "https://eu.api.blizzard.com/data/wow/realm/index?namespace=dynamic-eu&locale=en_GB&access_token="+token
    response = requests.get(search)
    return response.json()

def eu_connected_realms(realmID):
    search = "https://eu.api.blizzard.com/data/wow/connected-realm/"+str(realmID)+"?namespace=dynamic-eu&locale=en_GB&access_token="+token
    response = requests.get(search)
    return response.json()

In [ ]:
# Get Parent Realms
connected_realms = pd.DataFrame(columns=['id', 'has_queue', 'realms', '_links.self.href', 'status.type','status.name', 'population.type', 'population.name','mythic_leaderboards.href', 'auctions.href'])
eu_realms = pd.json_normalize(eu_realms(), record_path = ['realms'])
ids= list(eu_realms['id'])
for realmID in ids:
    data = eu_connected_realms(realmID)
    if data.get('code') != 404:
        temp = pd.json_normalize(data)
        connected_realms = connected_realms.append(temp)
connected_realms['parent_realm'] = connected_realms['realms'].apply(pd.Series)[0].apply(pd.Series)['name']
connected_realms.to_csv('parent_realms.csv',index=False)

In [ ]:
def get_leaderboard(realmID,dungeonID,periodID):
    search = "https://eu.api.blizzard.com/data/wow/connected-realm/"+str(realmID)+"/mythic-leaderboard/"+str(dungeonID)+"/period/"+str(periodID)+"?namespace=dynamic-eu&locale=en_GB&access_token="+token
    response = requests.get(search)
    if response.status_code == 200:
        return response.json()
    else:
        return

In [ ]:
# Read in Parent Realms
connected_realms = pd.read_csv('parent_realms.csv')

dungeons = [375,376,377,378,379,380,381,382]
realms = list(connected_realms['id'].unique())
period = 784 # Find out what this means, weekly reset? 
tank_spec = 66 # paladin tanks
i = 1
for realm in realms:
    full_leaderboard = pd.DataFrame(columns=['profile.id','profile.name','profile.realm.slug','specialization.id','faction.type'])
    print(str(i) + ' / ' + str(len(realms)))
    i+=1
    for dungeon in dungeons:
        temp = pd.json_normalize(get_leaderboard(realm,dungeon,period), record_path = ['leading_groups'])
        for group in temp['members']:
            paladins = pd.json_normalize(group)
            paladins = paladins[paladins['specialization.id']==tank_spec][['profile.id','profile.name','profile.realm.slug','specialization.id','faction.type']]
            full_leaderboard = full_leaderboard.append(paladins)
    full_leaderboard = full_leaderboard.drop_duplicates()
    full_leaderboard.to_csv('realms\\'+str(realm)+'.csv', index=False)

In [12]:
# PvP Leaderboards

def twoVtwo():
    # 2v2
    search1 = "https://eu.api.blizzard.com/data/wow/pvp-season/26/pvp-leaderboard/2v2?namespace=dynamic-eu&locale=en_GB&access_token="+token
    response = requests.get(search)
    return response.json()

def threeVthree():
    # 3v3
    search2 = "https://eu.api.blizzard.com/data/wow/pvp-season/27/pvp-leaderboard/3v3?namespace=dynamic-eu&locale=en_GB&access_token="+token
    response = requests.get(search)
    return response.json()

def rbg():
    # rgb
    search = "https://eu.api.blizzard.com/data/wow/pvp-season/27/pvp-leaderboard/rgb?namespace=dynamic-eu&locale=en_GB&access_token="+token
    response = requests.get(search)
    return response.json()


In [14]:
data = pd.json_normalize( twoVtwo() )
data

,code,type,detail
0,404,BLZWEBAPI00000404,Not Found
